In [225]:
import pandas as pd
import datetime
import numpy as np

In [226]:
df = pd.read_csv('dataset_visitors.csv')

    visit_datetime  – дата и время визита 
	referrer – ссылка перехода на сайт 
	visitor_device – устройство посетителя 
	is_lead – если True то посетитель оставил заявку во время ссесии


    Задания:
    1.	Нужно извлечь utm-метки из колонки referrer в итоге в датасете добавятся новых колонки: utm_source, utm_medium, utm_campaign, utm_term, utm_content с соответствующими значениями.
    2.	Внутри utm_content у некоторых ссылок есть параметр crid, за которым после символа | следует цифра (например: crid|17) – это номер рекламного баннера. Необходимо добавить колонку creative_id в датасет с этими номерами в сессиях в которых они есть.
    3.	В колонке visitor_device необходимо избавится от словарей, оставив только значение поля “value”.
    4.	Необходимо добавить колонку channel значение которой будет зависеть от utm_source и referrer:    
    a.	Если referrer пуст, тогда channel = “direct”
    b.	Если referrer = “https://www.google.com/” тогда channel = “organic”
    c.	Если utm_source не пуст то берем значение оттуда
    5.	Добавить колонку week_day в которую на основе даты и visit_date необходимо поместить день недели в формате: Monday, Tuesday..


In [227]:
df.head()

,visit_date,referrer,visitor_device,is_lead
0,2021-03-27 10:38:19.482951,NaN,"{""value"": ""PC"", ""value_id"": ""desktop""}",True
1,2021-03-10 12:50:36.680002,NaN,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",NaN
2,2021-03-05 19:53:09.428238,NaN,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN
3,2021-03-08 10:15:12.315905,NaN,"{""value"": ""PC"", ""value_id"": ""desktop""}",True
4,2021-03-17 13:25:55.861624,https://www.google.com/,"{""value"": ""PC"", ""value_id"": ""desktop""}",True


In [228]:
df.shape

(1258, 4)

In [229]:
df.referrer.unique()

array([nan, 'https://www.google.com/',
       'https://callgear.ae/vpbx/?utm_source=google&utm_medium=cpc&utm_campaign=GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246&utm_term=voip cloud&utm_content=tid|kwd-13704117858|cid|12187113246|aid|494770255120|gid|119774347129|pos||src|g_|dvc|c|reg|1000013|rin||oh|&cm_id=12187113246_119774347129_494770255120_kwd-13704117858_c__g_&gclid=CjwKCAjwg4-EBhBwEiwAzYAlsjqer99BdzB-UJ13uiAGOv0aGEsOmBHmvkQ0k-zGs6S4z_Z9-k3rjBoCl88QAvD_BwE',
       'https://callgear.ae/control-employees-lp-oae/missed-calls-lp-oae/?utm_source=facebook&utm_medium=cpm&utm_campaign=FB_UAE_ALL_CONVERSION_GN_AG_J35|23848562300580067&utm_content=cid|23848562300580067|gid|23848562301270067|aid|23848562671900067|crid|91|targid|8|sid|6|pid|1|jobs|35&cm_id=fb_23848562300580067_23848562301270067_23848562671900067&fbclid=PAAab3yYwVb2KQg64Q4x8SAolC6TlO3y3Mv5B5A4sYDmsjuAjK5eABGSnGsfY',
       'https://callgear.ae/vpbx/?utm_source=google&utm_medium=cpc&utm_campaign=GAW_UAE_DU_SEARCH_VP_TR_K

In [ ]:
# Бегло пробежавшись по списку, видим, что не во всех случаях есть все необходимые utm-метки

In [230]:
df_new = df.query('referrer.notnull()')

In [231]:
def def1(data):
    return data.replace('?','&')

In [232]:
df_new['referrer'] = df_new.referrer.apply(def1)
df_new = df_new.query('referrer != "https://www.google.com/"')

C:\Users\aliso\AppData\Local\Temp\ipykernel_1520\3553657898.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['referrer'] = df_new.referrer.apply(def1)


In [233]:
df_new.head()

# создали новый df со значениями в колонке referrer, которые нам необходимо обработать
# убрали пустые ячейки и ячейки с google.com
# оставили ссылки, в которых есть utm метки
# заменили ? на & для удобства сплита, так как первая метка отделяется "?" по умолчанию.

,visit_date,referrer,visitor_device,is_lead
7,2021-03-10 07:16:53.577870,https://callgear.ae/vpbx/&utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN
17,2021-03-12 16:38:31.863260,https://callgear.ae/control-employees-lp-oae/m...,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",True
39,2021-03-26 12:50:36.529962,https://callgear.ae/vpbx/&utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN
41,2021-03-18 08:23:59.871134,https://callgear.ae/pricing/&cm_id=12187113246...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN
44,2021-03-13 12:11:57.159766,https://callgear.ae/vpbx/&utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",True


In [234]:
df_new['col'] = df_new.referrer.str.split('&utm_')


# разделили значения колонок на нужные нам ячейки

In [235]:
df_new['utm_source'] = df_new['col'].str[1]
df_new['utm_medium'] = df_new['col'].str[2]
df_new['utm_campaign'] = df_new['col'].str[3]

df_new['utm_source'] = df_new['utm_source'].str.split('=').str[1]
df_new['utm_medium'] = df_new['utm_medium'].str.split('=').str[1]
df_new['utm_campaign'] = df_new['utm_campaign'].str.split('=').str[1]

df_new.head()

# Создаем требуемые колонки, обращаясь к элементам, которые создали в прошлой ячейке.
# Сделали сплит по '=' и обратились к элементу после '=', чтобы избавиться от названия метки в строках

# Проблема заключается в том, что после 'utm_campaign' в некоторых случаях идет 'utm_content', а в других 'utm_term'
# Использование предыдущего метода создаст в одной колонке перемешанные данные

,visit_date,referrer,visitor_device,is_lead,col,utm_source,utm_medium,utm_campaign
7,2021-03-10 07:16:53.577870,https://callgear.ae/vpbx/&utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246
17,2021-03-12 16:38:31.863260,https://callgear.ae/control-employees-lp-oae/m...,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",True,[https://callgear.ae/control-employees-lp-oae/...,facebook,cpm,FB_UAE_ALL_CONVERSION_GN_AG_J35|23848562300580067
39,2021-03-26 12:50:36.529962,https://callgear.ae/vpbx/&utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246
41,2021-03-18 08:23:59.871134,https://callgear.ae/pricing/&cm_id=12187113246...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN,[https://callgear.ae/pricing/&cm_id=1218711324...,google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246
44,2021-03-13 12:11:57.159766,https://callgear.ae/vpbx/&utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",True,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246


In [236]:
df_new['newcol'] = df_new.referrer.str.split('&utm_term=')
df_new['newcol'] = df_new['newcol'].str[1]
df_new['newcol'] = df_new.newcol.str.split('&utm_content=').str[0]
df_new = df_new.rename(columns={'newcol':'utm_term'})

# создаем колонку с разделением по '&utm_term='
# берем [1] элемент, чтобы вытащить значение utm_term
# в некоторых случаях в строке после utm_term идет utm_content
# делаем сплит и берем [0] элемент, чтобы оставить в колонке только значения utm_tem

In [237]:
df_new

,visit_date,referrer,visitor_device,is_lead,col,utm_source,utm_medium,utm_campaign,utm_term
7,2021-03-10 07:16:53.577870,https://callgear.ae/vpbx/&utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,voip cloud
17,2021-03-12 16:38:31.863260,https://callgear.ae/control-employees-lp-oae/m...,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",True,[https://callgear.ae/control-employees-lp-oae/...,facebook,cpm,FB_UAE_ALL_CONVERSION_GN_AG_J35|23848562300580067,NaN
39,2021-03-26 12:50:36.529962,https://callgear.ae/vpbx/&utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,voip cloud
41,2021-03-18 08:23:59.871134,https://callgear.ae/pricing/&cm_id=12187113246...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN,[https://callgear.ae/pricing/&cm_id=1218711324...,google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,system pabx
44,2021-03-13 12:11:57.159766,https://callgear.ae/vpbx/&utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",True,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,phone virtual
...,...,...,...,...,...,...,...,...,...
1245,2021-03-02 21:52:28.523109,https://callgear.ae/control-employees-lp-oae/m...,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",True,[https://callgear.ae/control-employees-lp-oae/...,facebook,cpm,FB_UAE_ALL_CONVERSION_GN_AG_J35|23848562300580067,NaN
1246,2021-03-25 23:08:05.975038,https://callgear.ae/landline-uae-number/up/&ut...,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",True,"[https://callgear.ae/landline-uae-number/up/, ...",facebook,cpm,FB_UAE_ALL_CONVERSION_GN_AG_J34|23848680267780067,NaN
1248,2021-03-24 01:04:22.813035,https://callgear.ae/missed-calls-lp-oae-2/&utm...,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",True,"[https://callgear.ae/missed-calls-lp-oae-2/, s...",facebook,cpm,FB_UAE_DU_CONVERSION_GN_AG_J35|23848827103510067,NaN
1250,2021-03-18 18:34:20.057726,https://callgear.ae/missed-calls-lp-oae-2/&utm...,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",True,"[https://callgear.ae/missed-calls-lp-oae-2/, s...",facebook,cpm,FB_UAE_DU_CONVERSION_GN_AG_J35|23848827103510067,NaN


In [238]:
df_new['utm_content'] = df_new.referrer.str.split('&utm_content=')
df_new['utm_content'] = df_new['utm_content'].str[1]
df_new['utm_content'] = df_new.utm_content.str.split('&utm_term=').str[0]

# создаем колонку с разделением по '&utm_content='
# берем [1] элемент, чтобы вытащить значение utm_content
# делаем проверку на наличие в строке 'utm_term' (мало ли)
# делаем сплит и берем [0] элемент, чтобы оставить в колонке только значения utm_content

In [239]:
df_new = df_new.drop(columns=['visit_date','referrer','visitor_device','is_lead'])
df_new.head()

,col,utm_source,utm_medium,utm_campaign,utm_term,utm_content
7,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,voip cloud,tid|kwd-13704117858|cid|12187113246|aid|494770...
17,[https://callgear.ae/control-employees-lp-oae/...,facebook,cpm,FB_UAE_ALL_CONVERSION_GN_AG_J35|23848562300580067,NaN,cid|23848562300580067|gid|23848562301270067|ai...
39,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,voip cloud,tid|kwd-13704117858|cid|12187113246|aid|494770...
41,[https://callgear.ae/pricing/&cm_id=1218711324...,google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,system pabx,tid|kwd-351055527567|cid|12187113246|aid|50919...
44,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,phone virtual,tid|kwd-299273384361|cid|12187113246|aid|50919...


In [240]:
df = df.merge(df_new, how='left', left_index=True, right_index=True)

# добавляем получившиеся колонки в нашу таблицу

In [241]:
df.head()

,visit_date,referrer,visitor_device,is_lead,col,utm_source,utm_medium,utm_campaign,utm_term,utm_content
0,2021-03-27 10:38:19.482951,NaN,"{""value"": ""PC"", ""value_id"": ""desktop""}",True,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-10 12:50:36.680002,NaN,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-05 19:53:09.428238,NaN,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-03-08 10:15:12.315905,NaN,"{""value"": ""PC"", ""value_id"": ""desktop""}",True,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-03-17 13:25:55.861624,https://www.google.com/,"{""value"": ""PC"", ""value_id"": ""desktop""}",True,NaN,NaN,NaN,NaN,NaN,NaN


In [242]:
df['creative_id'] = df.utm_content.str.split('crid').str[1]
df['creative_id'] = df.creative_id.str.split('|').str[1]

# создаем колонку creative_id, в которой будут значения из колонки utm_content после ключа 'crid'
# достаем из полученного значения числа после ключа '|'

In [243]:
df.head()

,visit_date,referrer,visitor_device,is_lead,col,utm_source,utm_medium,utm_campaign,utm_term,utm_content,creative_id
0,2021-03-27 10:38:19.482951,NaN,"{""value"": ""PC"", ""value_id"": ""desktop""}",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-10 12:50:36.680002,NaN,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-05 19:53:09.428238,NaN,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-03-08 10:15:12.315905,NaN,"{""value"": ""PC"", ""value_id"": ""desktop""}",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-03-17 13:25:55.861624,https://www.google.com/,"{""value"": ""PC"", ""value_id"": ""desktop""}",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
df['visitor_device'] = df.visitor_device.str.split(':').str[1]
df.visitor_device = df.visitor_device.str.split(',').str[0]

# сплитим значения колонки и вытаскиваем необходимое значение

In [245]:
df.head()

,visit_date,referrer,visitor_device,is_lead,col,utm_source,utm_medium,utm_campaign,utm_term,utm_content,creative_id
0,2021-03-27 10:38:19.482951,NaN,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-10 12:50:36.680002,NaN,"""Mobile""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-05 19:53:09.428238,NaN,"""PC""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-03-08 10:15:12.315905,NaN,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-03-17 13:25:55.861624,https://www.google.com/,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
df.dtypes

visit_date        object
referrer          object
visitor_device    object
is_lead           object
col               object
utm_source        object
utm_medium        object
utm_campaign      object
utm_term          object
utm_content       object
creative_id       object
dtype: object

In [247]:
df['visit_date'] = pd.to_datetime(df['visit_date'])
df['week_day'] = df['visit_date'].dt.day_name()

# переводим колонку 'visit_date' в формат даты
# создаем отдельную колонку 'week_day' с названием дня недели относительно колонки 'visit_date'

In [248]:
df.head()

,visit_date,referrer,visitor_device,is_lead,col,utm_source,utm_medium,utm_campaign,utm_term,utm_content,creative_id,week_day
0,2021-03-27 10:38:19.482951,NaN,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saturday
1,2021-03-10 12:50:36.680002,NaN,"""Mobile""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday
2,2021-03-05 19:53:09.428238,NaN,"""PC""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Friday
3,2021-03-08 10:15:12.315905,NaN,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday
4,2021-03-17 13:25:55.861624,https://www.google.com/,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday


In [249]:
df['channel'] = df.referrer.isnull()
df.loc[(df.referrer == 'NaN'), ('channel')] = 'direct'
df['channel'] = np.where((df.referrer.isnull()), 'direct', df.channel)
df['channel'] = np.where((df.referrer == 'https://www.google.com/'), 'organic', df.channel)

# добавляем условия для значений колонки 'channel' относительно других колонок

In [250]:
df.head(20)

,visit_date,referrer,visitor_device,is_lead,col,utm_source,utm_medium,utm_campaign,utm_term,utm_content,creative_id,week_day,channel
0,2021-03-27 10:38:19.482951,NaN,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saturday,direct
1,2021-03-10 12:50:36.680002,NaN,"""Mobile""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,direct
2,2021-03-05 19:53:09.428238,NaN,"""PC""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Friday,direct
3,2021-03-08 10:15:12.315905,NaN,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday,direct
4,2021-03-17 13:25:55.861624,https://www.google.com/,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,organic
5,2021-03-23 10:38:49.041414,NaN,"""PC""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tuesday,direct
6,2021-03-28 23:16:19.100134,NaN,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday,direct
7,2021-03-10 07:16:53.577870,https://callgear.ae/vpbx/?utm_source=google&ut...,"""PC""",NaN,"[https://callgear.ae/vpbx/, source=google, med...",google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,voip cloud,tid|kwd-13704117858|cid|12187113246|aid|494770...,NaN,Wednesday,False
8,2021-03-23 20:03:21.098973,https://www.google.com/,"""PC""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tuesday,organic
9,2021-03-04 23:16:36.637201,NaN,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,direct


In [251]:
df['channel'] = np.where((df.utm_source.notnull()), df.utm_source, df.channel)
# добавляем условия для значений колонки 'channel' относительно 'utm_source'

In [261]:
df = df.drop(columns='col')
# избавляемся от лишних колонок

In [262]:
df.head()

,visit_date,referrer,visitor_device,is_lead,utm_source,utm_medium,utm_campaign,utm_term,utm_content,creative_id,week_day,channel
0,2021-03-27 10:38:19.482951,NaN,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,Saturday,direct
1,2021-03-10 12:50:36.680002,NaN,"""Mobile""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,direct
2,2021-03-05 19:53:09.428238,NaN,"""PC""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Friday,direct
3,2021-03-08 10:15:12.315905,NaN,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,Monday,direct
4,2021-03-17 13:25:55.861624,https://www.google.com/,"""PC""",True,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,organic


In [264]:
df.to_csv('final.csv')
# сохраняем csv файл